# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL"}

> Entering new CrewAgentExecutor chain...
I should start by reading the latest news and updates on AAPL to understand the current market sentiment and any potential factors affecting its stock price.

Action: Read website content
Action Input: {"website_url": "https://www.market

Final Answer: 
To successfully monitor and analyze market data for AAPL, I can use the Yahoo Finance website, specifically the AAPL stock page that provides comprehensive information about Apple Inc., including stock price, news, historical data, performance metrics, and more.

> Finished chain.
 

To successfully monitor and analyze market data for AAPL, I can use the Yahoo Finance website, specifically the AAPL stock page that provides comprehensive information about Apple Inc., including stock price, news, historical data, performance metrics, and more.

Final Answer: 
Insights and alerts about significant market opportunities or threats for AAPL can be obtained by continuously monitoring and analyzing market data for AAPL using statistical modeling and machine learning. To successfully monitor and analyze market data for AAPL, the Data Analyst can use the Yahoo Finance website, specifically the AAPL stock page that provides comprehensive information about Apple Inc., including stoc

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Calendar Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Mat

Final Answer: Apple Inc. (AAPL) is a leading consumer electronics company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products, including the iPhone, Mac, iPad, and various wearable devices. Apple also provides services such as AppleCare support, cloud services, Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, Apple Card, and Apple Pay. The company serves consumers, small and mid-sized businesses, as well as education, enterprise, and government markets. Apple distributes third-party applications through the App Store and sells its products through retail and online stores, as well as third-party channels. As of May 22, 2024, Apple's stock price closed at $190.90, with a market cap of $2.927 trillion. The company has a strong financial performance, with a profit margin of 26.31%, return on assets of 22.07%, return on equity of 147.25%, and revenue of $381.62 billi

Thought: Now that the Trading Strategy Developer has analyzed approved trading strategies for AAPL, I need to ask the Data Analyst a question about the current market conditions for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "What are the current market conditions for AAPL?", "context": "Apple Inc. (AAPL) is a leading consumer electronics company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products, including the iPhone, Mac, iPad, and various wearable devices. Apple also provides services such as AppleCare support, cloud services, Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, Apple Card, and Apple Pay. As of May 22, 2024, Apple's stock price closed at $190.90, with a market cap of $2.927 trillion. The company has a strong financial performance, with a profit margin of 26.31%, return on assets of 22.07%, retu

Final Answer: 
Apple Inc. (AAPL) closed at $190.90 on May 22, 2024, with a market cap of $2.927 trillion. The company has a profit margin of 26.31%, return on assets of 22.07%, return on equity of 147.25%, and revenue of $381.62 billion. AAPL stock has delivered positive total returns compared to the S&P 500 index over various time periods. Overall, Apple Inc. remains a key player in the technology sector with a strong financial performance and continues to innovate and expand its product and service offerings.

> Finished chain.
 

Apple Inc. (AAPL) closed at $190.90 on May 22, 2024, with a market cap of $2.927 trillion. The company has a profit margin of 26.31%, return on assets of 22.07%, return on equity of 147.25%, and revenue of $381.62 billion. AAPL stock has delivered positive total returns compared to the S&P 500 index over various time periods. Overall, Apple Inc. remains a key player in the technology sector with a strong financial performance and continues to innovate and e

Thought: 
Based on the search results, it is clear that a short position in a put option carries considerable downside risk, with potential for unlimited losses. It is essential to have enough funds in the account to cover potential losses in case the option is assigned.

Final Answer:
After analyzing the risks associated with a short position in put options for AAPL, it is evident that this strategy can lead to significant losses and should be approached with caution. The potential for unlimited liability and the need for sufficient funds to cover potential losses make this strategy risky and unsuitable for all investors. It is crucial to carefully assess the risk tolerance and financial capabilities before considering a short position in put options for AAPL.

> Finished chain.
 

After analyzing the risks associated with a short position in put options for AAPL, it is evident that this strategy can lead to significant losses and should be approached with caution. The potential for u

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

After evaluating the risks associated with the proposed trading strategies and execution plans for AAPL, it is crucial to consider the potential challenges and develop mitigation strategies to minimize risks and maximize gains. 

One of the risks identified is the use of a short position in put options with a strike price below the current market price of AAPL shares. This strategy can lead to significant losses due to the potential for unlimited liability and the need for sufficient funds to cover potential losses. To mitigate this risk, it is recommended to carefully assess risk tolerance, set stop-loss orders, and diversify the portfolio to spread out risks.

Another risk to consider is the implementation of stock option trading strategies such as Straddle, Buy-Write, Risk Reversal, and Call Spread. While these strategies can provide insights into potential gains and losses, they also come with risks such as market volatility, unexpected price movements, and liquidity issues. To mitigate these risks, it is advised to conduct thorough research, stay informed about market conditions, and consider using risk management tools such as options spreads and collars.

Day trading strategies for Apple stocks involve analyzing pre-market prices, preparing charts, and conducting technical analysis of AAPL stock charts, market data, and options flow. While these strategies can be lucrative, they also carry risks such as market fluctuations, timing issues, and emotional trading decisions. To mitigate these risks, it is recommended to set clear trading goals, follow a disciplined trading plan, and use risk management techniques such as position sizing and leverage control.

In conclusion, by identifying potential risks associated with the proposed trading strategies and execution plans for AAPL and implementing appropriate mitigation strategies, traders can navigate the market with confidence and increase the likelihood of successful trades. It is essential to stay informed, adapt to changing market conditions, and continuously evaluate and adjust trading strategies to achieve long-term success in trading AAPL.